<a href="https://colab.research.google.com/github/SebastianWikt/DS4002-P3/blob/main/Scripts/ds4002p3movieanalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
#rm -r DS4002-P3/

In [43]:
! git clone https://github.com/SebastianWikt/DS4002-P3.git

fatal: destination path 'DS4002-P3' already exists and is not an empty directory.


In [44]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import cv2

def loadData(file):
  # The column names are adopted from the dataset page on Kaggle
  df = pd.read_csv(file)
  return df

def plot_color_histograms(image_paths):
    for img_path in image_paths:
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        colors = ('r', 'g', 'b')
        plt.figure(figsize=(10, 5))
        plt.title(f'Color Histogram for {os.path.basename(img_path)}')

        for i, color in enumerate(colors):
            hist = cv2.calcHist([img], [i], None, [256], [0, 256])
            plt.plot(hist, color=color)
            plt.xlim([0, 256])
        plt.xlabel('Pixel Intensity')
        plt.ylabel('Frequency')
        plt.show()

def check_color_channels(image_paths):
    for img_path in image_paths:
        img = cv2.imread(img_path)
        if len(img.shape) == 2:  # Grayscale images have 2 dimensions
            print(f'{os.path.basename(img_path)} is Grayscale')
        elif len(img.shape) == 3 and img.shape[2] == 3:  # RGB images have 3 dimensions
            print(f'{os.path.basename(img_path)} is RGB')
        else:
            print(f'{os.path.basename(img_path)} has an unusual format')

def check_image_quality(image_paths, min_resolution=(150, 150)):
    for img_path in image_paths:
        img = cv2.imread(img_path)
        # Check resolution
        height, width = img.shape[:2]
        if width < min_resolution[0] or height < min_resolution[1]:
            print(f'Low-resolution image: {os.path.basename(img_path)} - {width}x{height} pixels')

        # Check brightness (mean of pixel intensities)
        brightness = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY).mean()
        print(f'{os.path.basename(img_path)} - Mean Brightness: {brightness:.2f}')

        # Check blur using Laplacian variance
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        laplacian_var = cv2.Laplacian(gray, cv2.CV_64F).var()
        if laplacian_var < 100:  # Threshold for blur detection (adjust as needed)
            print(f'Blurred image: {os.path.basename(img_path)} - Variance: {laplacian_var:.2f}')

def get_kernel_dimensions(version, shape, divisor):
    image_width = shape[1]

    # original
    if version == 1:
        return 3, 3

    # square 10% width
    if version == 2:
        return int(0.1 * image_width / divisor), int(0.1 * image_width / divisor)

    # square 20% width
    if version == 3:
        return int(0.2 * image_width / divisor), int(0.2 * image_width / divisor)

In [45]:
df = pd.read_csv('/content/DS4002-P3/Data/MoviePosterData.csv')
genre_horror = df[df['Genre'] == 'Horror']
genre_romance = df[df['Genre'] == 'Romance']
genre_comedy = df[df['Genre'] == 'Comedy']

In [46]:
import os
import requests
from PIL import Image, ImageStat
from io import BytesIO
import numpy as np
import cv2

folder_path = '/content/images'
os.makedirs(folder_path, exist_ok=True)

image_paths = []

for i, url in enumerate(df['MoviePoster']):
    try:
        response = requests.get(url)
        image = Image.open(BytesIO(response.content))

        image = image.resize((106, 161), Image.Resampling.LANCZOS)


        image_filename = f"image_{i}.jpg"
        image_path = os.path.join(folder_path, image_filename)
        image.save(image_path)

        img = Image.open(image_path)

        # Get image size (width, height)
        width, height = img.size
        print(f"Image size: {width}x{height}")

        image_paths.append(image_path)

        # print(f"Saved image_{i}.jpg")
    except Exception as e:
        print(f"Failed to download {url}: {e}")
        image_paths.append(np.NaN)

df['LocalImagePath'] = image_paths

# Save the updated DataFrame back to a CSV
updated_csv_path = '/content/updated_movie_posters.csv'
df.to_csv(updated_csv_path, index=False)

Image size: 106x161
Image size: 106x161
Image size: 106x161
Image size: 106x161
Image size: 106x161
Image size: 106x161
Image size: 106x161
Image size: 106x161
Image size: 106x161
Image size: 106x161
Image size: 106x161
Image size: 106x161
Image size: 106x161
Image size: 106x161
Image size: 106x161
Image size: 106x161
Image size: 106x161
Image size: 106x161
Image size: 106x161
Image size: 106x161
Image size: 106x161
Image size: 106x161
Image size: 106x161
Image size: 106x161
Image size: 106x161
Image size: 106x161
Image size: 106x161
Image size: 106x161
Image size: 106x161
Image size: 106x161
Image size: 106x161
Image size: 106x161
Image size: 106x161
Image size: 106x161
Image size: 106x161
Image size: 106x161
Image size: 106x161
Image size: 106x161
Image size: 106x161
Image size: 106x161
Image size: 106x161
Image size: 106x161
Image size: 106x161
Image size: 106x161
Image size: 106x161
Image size: 106x161
Image size: 106x161
Image size: 106x161
Image size: 106x161
Image size: 106x161


In [51]:
import os
import os
import shutil

# Organize images into subdirectories by Genre
for _, row in df.iterrows():
    genre = row['Genre']
    img_path = row['LocalImagePath']

    genre_folder = os.path.join(folder_path, genre)
    os.makedirs(genre_folder, exist_ok=True)
    if os.path.exists(img_path):
        shutil.move(img_path, os.path.join(genre_folder, os.path.basename(img_path)))
print(os.listdir('/content/images'))  # List all files in the directory


TypeError: stat: path should be string, bytes, os.PathLike or integer, not float

In [47]:
from sklearn.model_selection import train_test_split

# Split the dataset
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

train_labels = train_df['Genre']
test_labels = test_df['Genre']

train_images = train_df['LocalImagePath']
test_images = test_df['LocalImagePath']

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    folder_path,
    validation_split = 0.2,
    subset = "training",
    seed = 42,
    image_size= (106, 161),
    batch_size = 32
)

print(train_df.head())

# Save the train and test DataFrames as separate CSV files
train_csv_path = '/content/train_movie_posters.csv'
test_csv_path = '/content/test_movie_posters.csv'

train_df.to_csv(train_csv_path, index=False)
test_df.to_csv(test_csv_path, index=False)

Found 0 files belonging to 0 classes.
Using 0 files for training.


ValueError: No images found in directory /content/images. Allowed formats: ('.bmp', '.gif', '.jpeg', '.jpg', '.png')

In [ ]:
### GRAPHS FROM FULL DATA
image_folder = folder_path
image_paths = [os.path.join(image_folder, img) for img in os.listdir(image_folder)]
# plot_color_histograms(image_paths)

In [ ]:
# Calculating average image r,g,b and adding to dataframe
rgb_values = []
for i in range(550):
    if i == 444 or i == 496:
      rgb_values.append(np.NaN)
    else:
      img = Image.open(f'/content/images/image_{i}.jpg')
      img = img.convert('RGB')  # Ensure image is in RGB format
      img_array = np.array(img)
      rgb_values.append(img_array.mean(axis=(0, 1)))  # Calculate mean RGB values for each image
      df.loc[i, 'avg_r'] = rgb_values[i][0]
      df.loc[i, 'avg_g'] = rgb_values[i][1]
      df.loc[i, 'avg_b'] = rgb_values[i][2]

df.head()

In [ ]:
# Plotting a histogram of rgb values for each genre
for genre in df['Genre'].unique():
  genre_df = df[df['Genre'] == genre]
  plt.figure(figsize=(10, 5))
  plt.title(f'RGB Histogram for {genre} Genre')
  plt.hist(genre_df['avg_r'], bins=20, color='red', alpha=0.5, label='Red')
  plt.hist(genre_df['avg_g'], bins=20, color='green', alpha=0.5, label='Green')
  plt.hist(genre_df['avg_b'], bins=20, color='blue', alpha=0.5, label='Blue')
  plt.xlabel('RGB Value')
  plt.ylabel('Frequency')
  plt.legend()
  plt.show()

In [ ]:
# Calculating average image brightness and adding to dataframe
def brightness( im_file ):
   im = Image.open(im_file).convert('L')
   stat = ImageStat.Stat(im)
   return stat.mean[0]

brightness_values = []

for i in range(550):
    if i == 444 or i == 496:
      brightness_values.append(np.NaN)
    else:
      image = f'/content/images/image_{i}.jpg'
      pbrightness = brightness(image)
      brightness_values.append(pbrightness)


df["brightness"] = brightness_values

df.head()

In [ ]:
# Plotting a histogram of brightness grouped by genre

plt.figure(figsize=(10, 6))

sns.histplot(df[df['Genre'] == 'Horror']['brightness'], color='black', label='Horror', kde=True)
sns.histplot(df[df['Genre'] == 'Romance']['brightness'], color='red', label='Romance', kde=True)
sns.histplot(df[df['Genre'] == 'Comedy']['brightness'], color='yellow', label='Comedy', kde=True)


plt.xlabel('Brightness')
plt.ylabel('Frequency')
plt.title('Image Brightness by Genre')
plt.legend()
plt.show()

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# 140 x 207 to 106 x 161

num_classes = 3
kernel_dimensions1 = (3, 3)
kernel_dimensions2 = (2, 2)

model = Sequential([
      Conv2D(32, kernel_dimensions1, padding='same', input_shape=(106, 161, 3), activation='relu'),
      Conv2D(32, kernel_dimensions1, activation='relu'),
      MaxPooling2D(pool_size=(2, 2)),
      Dropout(0.25),

      Conv2D(64, kernel_dimensions2, padding='same', activation='relu'),
      Conv2D(64, kernel_dimensions2, activation='relu'),
      MaxPooling2D(pool_size=(2, 2)),
      Dropout(0.25),

      Flatten(),
      Dense(512, activation='relu'),
      Dropout(0.5),
      Dense(num_classes, activation='softmax')
  ])

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
print(type(train_images), train_images.dtype)  # Should be <class 'numpy.ndarray'> and dtype should be float32
print(type(train_labels), train_labels.dtype)

In [ ]:
history = model.fit(
    train_images,
    train_labels,
    epochs=10,                # Adjust epochs based on your dataset
    batch_size=32,            # Common batch size, can be adjusted
    validation_data=(test_images, test_labels)  # Monitor performance on validation data
)